In [ ]:
# imports
import requests
import json
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import inspect

# settings to load data
mypath = '/Users/nancynyambura/Desktop/Digital Shaper Program/ScrumBusters/Frame 1/Telegram/data/04_raw_financial/'

onlyfiles = [mypath + f for f in listdir(mypath) if (isfile(join(mypath, f)))]

onlyfiles = [f for f in onlyfiles if 'simplified' in f ]


In [ ]:
#%cd G:\Meine Ablage\TechLabs\Scam Busters
%cd /Users/nancynyambura/Desktop/Digital Shaper Program

In [ ]:
time_to_pump = 5
expected_pump_row = -30

In [ ]:
max_possible_return = test_df.iloc[-30:-25].high.max() / test_df.iloc[-40:-31].low.min()

In [ ]:
test_df = pd.read_csv(onlyfiles[7],index_col=0)
test_df['open_time'] = pd.to_datetime(test_df['open_time'],unit='ms') # turnes unix time to datetime
idxmax = (test_df.high/test_df.open).idxmax()
print(idxmax,(test_df.high/test_df.open)[idxmax])

In [ ]:
test_df.n_trades.plot()

In [ ]:
test_df.high.plot()

In [ ]:
(test_df.high.rolling(5).max() / test_df.high.rolling(5).min()).plot()

In [ ]:
def pump_success_indicator(file):
    df = pd.read_csv(file)
    high, open = df.high, df.open
    idx_indicator = (high/open).idxmax()
    indicator = (high/open)[idx_indicator]
    high_max=df['high'].rolling(2).max()
    open_max=df['open'].rolling(2).max()
    open_max = open_max.dropna().reset_index(drop=True)
    max_possible_return = df.iloc[-40:-25].high.max() / df.iloc[-40:-25].low.min()
    max_5m_return = (high_max/open_max).dropna().max()
    max_5m_return_idx = (high_max/open_max).dropna().idxmax()
    return {'indicator':indicator, 'idx_indicator':idx_indicator,
        'max_possible_return':max_possible_return}

all_indicators = pd.DataFrame([
    pump_success_indicator(file)
    for file
    in onlyfiles
])

In [ ]:
all_indicators.sort_values('max_possible_return').iloc[50:]

In [ ]:
test_df = pd.read_csv(onlyfiles[10])
test_df.open.plot()

In [ ]:
# cut dataframe to end just before the pump
def get_prepump_df(df):
    ()

In [ ]:
def returns_before_pump(df,hours_to_pump):
    '''returns the ROI if investing hours_to_pump 
    number of hours before the exit and exiting
    time_to_pump minutes before the pump'''
    
    return 1

In [ ]:
#SimpleFeatures



def summe(df,cols):
    res = [
        df[col].sum()
        for col in cols
    ]
    return res

def durchsch(df,cols):
    res = [
    df[col].mean()
    for col in cols
]
    return res

def maximum(df,cols):
    res = [
        df[col].max()
        for col in cols
    ]
    return res

# logarithm of column^^
def logarithmus(df,cols):
    res = [
        df[col].log()
        for col in cols
    ]
    return res

# daily return
def verhaeltnis(df,cols):
    res = [
    (df[col]/df[cols])-1
    for col in cols
    ]
    return res

def schnitt(df,cols):
    res = [
        df[col].mean() 
            for col in cols
    ]
    return res


def standard(df,cols):
    res = [
        df[col].std() 
            for col in cols
    ]
    return res

# maximum difference between 
def m_potential(df):
    res = (df['high']/df['open'])
    return res

#a prepump ?
def will_pump(df,cols,s_val): # cut = -31 for pre pump data, s_val has range between 1-3
    sigma = s_val*standard(df,cols) + schnitt(df,cols)
    if df[cols].max()>= sigma:
        return True
    else :
        return False
    
def is_pump(df,cols,s_val): # cut = -31 for pre pump data, s_val has range between 1-3
    df =df.iloc[:] 
    sigma = s_val*standard(df,cols) + schnitt(df,cols)
    if df[cols].max()>= sigma:
        return True
    else :
        return False

def start_bot(df):
    if will_pump() == True:
        t = df.open_time()       # hier soll der Zeitpunkt des ersten Wertes
        range_t = (-31) - t      # als t gespeicehrt werden, der den Sigma wert überschreitet
    return  range_t               
    

def trade_per_vol(df):
    a = df.n_trades.sum()
    b = df.volume.sum()
    c = a/b
    return c

#def admins_budget(df):  # could be an indicator of how much profit is expected
    # time_range = start_bot(df) - 31
    # sum of buyers volume
    # may indicate roi in percent.
    
# Im Intervall der Abweichung +- 3*Sigma
# vom Erwartungswert sind 99,73 % aller Messwerte zu finden.
# trading win without participating in the pump event. Buy 1 hour before and sell 2h after pump for eg


# more complex features and combining them to higher lvl features
#marcetcap data


#will_pump(df,cols=['volume'],s_val=3)

In [ ]:
# full engineering for a single df
def compute_features(df):
    features = {
        'f01':summe(df,df.volume),
        'f02':maximum(df,df.volume),
        'f03':standard(df,df.volume),
        'f04':durchsch(df.df.volume),
        'f04':standard(df,df.n_trades),
    }
    return features

In [ ]:
# get all features
engineered_df = pd.concat([
    compute_features(pd.read_csv(file))
    for file
    in onlyfiles
])

In [ ]:
# add the label
# pump column
pump_column = pd.read_csv('/Users/nancynyambura/Desktop/Digital Shaper Program/ScrumBusters/Frame 1/Telegram/data/label_dataset - [filename_suspicious_messages].csv')
pump_column['datetime'] = pump_column['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
engineered_df = pd.merge(engineered_df, pump_column, left_on= 'open_time', right_on='datetime', how = 'left').drop(['Unnamed: 0', 'symbol'], axis = 1)



In [ ]:
'''# add market cap
market_cap_df = pd.read_csv('Frame 1\General history Data\Feature Engineering\market_cap.csv')
engineered_df = pd.merge(engineered_df, market_cap_df, left_on= 'coinnames', right_on='symbol', how = 'left').drop(['Unnamed: 0', 'symbol'], axis = 1)
engineered_df.market_cap = engineered_df.market_cap.fillna(market_cap_df.market_cap.min()*0.001)'''

In [ ]:
engineered_df.to_csv('Samples/financial_data/engineered/sample.csv')